# 概要

本ファイルは、学習済みモデルを使用して、3D物体検出モデルの推論を実行します。 

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output
本ファイルを実行することで、`results`ディレクトリに、推論結果(result.pkl)を生成します。

```
└ second.pytorc
     └ checkpoints
          └ {model_dir}
                ├ results    <- 本ファイルにて生成されます
                   └ step_xxxx    
                       └─ result.pkl
```

# 準備

あらかじめ、 `data`ディレクトリに、KITTIデータセット、`velodyne_painted_reduced`ディレクトリ、各メタ情報(pkl)ファイル、および、`second.pytorch/checkpoints/{model_dir}` ディレクトリに学習済みモデルがあることを確認してください。

> 本ファイルのデフォルトでは、公開の学習済みモデル`second.pytorch/checkpoints/pointpainting`を設定しています。


```
├data
   └ kitti   
       └ training
           ├─ image_2
           ├─ calib
           ├─ label_2
           ├─ velodyne
           ├─ velodyne_painted     <-- 1-1_painting_kitti.ipynb にて生成
           └─ velodyne_painted_reduced  <-- 1-2_create_dataset_kitti.ipynb にて生成
       ├ gt_database                <-- 1-2_create_dataset_kitti.ipynb にて生成
       ├ kitti_dbinfos_train.pkl    <-- 1-2_create_dataset_kitti.ipynb にて生成
       ├ kitti_infos_train.pkl      <-- 1-2_create_dataset_kitti.ipynb にて生成
       ├ kitti_infos_val.pkl        <-- 1-2_create_dataset_kitti.ipynb にて生成
       └ kitti_infos_trainval.pkl   <-- 1-2_create_dataset_kitti.ipynb にて生成

├second.pytorch
     └─ checkpoints
          └ {model_dir}
                ├── checkpoints.json
                ├── {model-step}.tckpt <-- 学習済みモデル
```

# 推論・評価

検証データ(val)に対して推論と評価を実施します。  
検証データの対象は`second.pytorch/second/data/ImageSets/val.txt`で定義されています。

In [1]:
# 設定
config_path = "../second.pytorch/second/configs/all.fhd_painted.config"

model_dir = "../second.pytorch/checkpoints/pointpainting"

In [2]:
!python ../second.pytorch/second/pytorch/train.py evaluate \
        --config_path=$config_path \
        --model_dir=$model_dir --measure_time=True --batch_size=4

[  41 1280 1056]
Restoring parameters from /work/second.pytorch/checkpoints/pointpainting/voxelnet-18570.tckpt
feature_map_size [1, 160, 132]
remain number of infos: 3769
Generate output labels...
[100.0%][===================>][3.18it/s][09:14>00:00]   
generate label finished(6.79/s). start eval:
avg example to torch time: 19.429 ms
avg prep time: 77.472 ms
avg voxel_feature_extractor time = 3.219 ms
avg middle forward time = 275.297 ms
avg rpn forward time = 150.431 ms
avg predict time = 62.081 ms
/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 108 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 117 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: Numb

## （オプション）推論結果の確認

推論データは pklファイルに保存されています。pklファイルを読み込みことで推論データを確認することができます。

In [3]:
import pickle

result_pkl = "../second.pytorch/checkpoints/pointpainting/eval_results/step_18570/result.pkl"

with open(result_pkl, 'rb') as f:
    result = pickle.load(f)

print("result size: ", len(result))
print(result[0]["box3d_lidar"])  # (7, n)  [x, y, z, l, h, w, rz]

result size:  3769
tensor([[ 4.6201e+01, -4.6468e+00,  2.9013e-02,  5.0090e-01,  1.7551e+00,
          1.7293e+00,  1.2575e+00],
        [ 1.5356e+01, -7.4090e+00,  3.4037e-02,  4.4621e-01,  1.5843e+00,
          1.6028e+00,  5.3754e-01],
        [ 4.6764e+01,  2.0221e+01, -1.5747e+00,  1.6073e+00,  4.1575e+00,
          1.5082e+00,  2.9629e+00]], device='cuda:0')


以上